In [1]:
import pandas as pd

In [15]:
def cped_get_personailty_description(r, p, p_or_n):
    character = 'Speaker'
    label = r['labels']
    key = p+ p_or_n
    # if label == 1:
    #     key = p+ '_pos'
    # else:
    #     key = p+ '_neg'
    return character + ' ' + personality_description[key]


def cped_get_emotion_prompt(r):
    dialog_state = eval(r['dialog_state'])
    emotions = eval(r['emotions'])
    character = 'Speaker'
    ans = ''
    for i in range(len(dialog_state)):
        if i == 0:
            if dialog_state[i] == 0:
                ans += 'First, the emotion of others is ' + emotions[i] + ', '
            elif dialog_state[i] == 1:
                ans += 'The emotion of ' + character + ' is initially ' + emotions[i] + ', '
        else:
            if dialog_state[i] == 0:
                ans += 'the emotion of others is ' + emotions[i] + ', '
            elif dialog_state[i] == 1:
                ans += character + ' respond with ' + emotions[i] + ', '
    return ans

In [3]:
personality_description = {
    'A_pos': 'is friendly, cooperative, empathetic, and compassionate, often prioritizing harmonious relationships and the well-being of others.',
    'A_neg': 'is confrontational, uncooperative, lacking empathy, and often prioritizing their own needs and desires over the well-being of others.',
    'C_pos': 'is organized, responsible, diligent, detail-oriented, and committed to achieving their goals with a strong sense of duty and self-discipline.',
    'C_neg': 'is disorganized, careless, impulsive, lacking discipline, and often displaying a disregard for responsibilities and commitments.',
    'E_pos': 'is outgoing, sociable, energetic, and thriving in social interactions, often seeking stimulation and enjoying the company of others.',
    'E_neg': 'is introverted, reserved, quiet, and often preferring solitude or smaller social settings, conserving energy and finding fulfillment in introspection and reflection.',
    'O_pos': 'has curiosity, open-mindedness, creativity, tolerance, emotional expressiveness, and willingness to embrace new experiences and ideas.',
    'O_neg': 'is closed-minded, resistant to change, lacking curiosity, intolerant of differences, emotionally guarded, and hesitant to explore new ideas or experiences.',
    'N_pos': 'is prone to experiencing negative emotions, such as anxiety, worry, and mood swings, often displaying heightened sensitivity to stress and a tendency towards self-doubt and emotional instability.',
    'N_neg': 'is emotionally stable, resilient, and composed, often displaying a calm and balanced demeanor, and having a tendency to handle stress and adversity with ease.'
}

In [19]:
SEED_list = [0,42,3407,1,2,3,4,5,6,7]
from sklearn.model_selection import train_test_split

for flow_len in [0.25, 0.5, 0.75, 1]:
    personality = ['A', 'C', 'E', 'O', 'N']
    for p in personality:
        print(p, '...')
        df = pd.read_csv('CPED_'+p+'_with_role.tsv', sep='\t')
        df_tmp = df
        
        
        df_tmp['pos_personality_description'] = df_tmp.apply(cped_get_personailty_description, p=p, p_or_n='_pos', axis=1)
        df_tmp['neg_personality_description'] = df_tmp.apply(cped_get_personailty_description, p=p, p_or_n='_neg', axis=1)
    
        df_tmp['personality_description'] = df_tmp['pos_personality_description']
            
        df_tmp['label'] = df_tmp['labels']
        df_tmp['nli_label'] = df_tmp['labels']
        
        df_tmp['affective_prompt'] = df_tmp.apply(cped_get_emotion_prompt, axis=1)
        df_tmp['sent'] = df_tmp['origin_sent']
        df['sent_and_prompt'] = df['origin_sent'] + df['affective_prompt']

        for seed in SEED_list:
            train_df, test_df = train_test_split(df_tmp, test_size=0.2, random_state=seed)
            valid_df, test_df = train_test_split(test_df, test_size=0.5, random_state=seed)
        
            train_df.to_csv('CPED/CPED_'+p+'_with_role_'+str(flow_len)+'_'+str(seed)+'_train.tsv', sep='\t', index=False)
            valid_df.to_csv('CPED/CPED_'+p+'_with_role_'+str(flow_len)+'_'+str(seed)+'_valid.tsv', sep='\t', index=False)
            test_df.to_csv('CPED/CPED_'+p+'_with_role_'+str(flow_len)+'_'+str(seed)+'_test.tsv', sep='\t', index=False)
        
        # df_tmp.to_csv('CPED_'+p+'_with_role_'+str(flow_len)+'.tsv', sep='\t', index=False)

A ...
C ...
E ...
O ...
N ...
A ...
C ...
E ...
O ...
N ...
A ...
C ...
E ...
O ...
N ...
A ...
C ...
E ...
O ...
N ...
